In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # default='warn'
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cs
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
df = pd.read_csv('1.csv')

In [2]:
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

In [3]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))

In [4]:
df['addr'] = df['neighbourhood'] +' '+ df['location']

In [5]:
df.set_index('name', inplace=True)
indices = pd.Series(df.index)

In [6]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_cuisine_matrix = tfidf.fit_transform(df['cuisines'])
tfidf_location_matrix = tfidf.fit_transform(df['addr'])

In [7]:
cuisine_similarity = cs(tfidf_cuisine_matrix,tfidf_cuisine_matrix)
location_similarity = cs(tfidf_location_matrix,tfidf_location_matrix)

In [8]:
def recommend(name):
    recommendations = []

    idx = indices[indices == name].index[0]

    cuisine_series = pd.Series(cuisine_similarity[idx]).sort_values(ascending=False)
    #print(cuisine_series)
    top_cuisine = list(cuisine_series.iloc[0:2000].index)

    location_series = pd.Series(location_similarity[idx]).sort_values(ascending=False)
    top_location = list(location_series.iloc[0:1000].index)

    for i in top_cuisine:
        for j in top_location:
            if i == j:
                recommendations.append(list(df.index)[i])

    df_new = pd.DataFrame(columns=['cuisines','rate', 'approx_cost', 'addr'])
    
    for each in recommendations:
        df_new = df_new.append(pd.DataFrame(df[['cuisines','rate', 'approx_cost', 'addr']][df.index == each].sample()))
    
    df_new = df_new.drop_duplicates()

    df_new['cost_diff'] = df_new['approx_cost'] - (df[df.index == name]['approx_cost'].values[0])
    drop_names = df_new[(df_new['cost_diff'] > 201) | (df_new['cost_diff'] < -201)].index
    df_new.drop(drop_names, inplace=True)

    df_new = df_new.sort_values(by='rate', ascending=False)

    df_new.to_csv('temp.csv')
    df_new.drop(['cost_diff'], axis=1, inplace=True)
    return df_new.head(10)

In [9]:
recommend('Jalsa')

,cuisines,rate,approx_cost,addr
Empire Restaurant,North Indian Mughlai South Indian Chinese,4.4,750.0,Banashankari Jayanagar
Meghana Foods,Biryani North Indian Chinese,4.4,600.0,Banashankari Jayanagar
The Royal Corner - Pai Viceroy,North Indian Chinese,4.2,900.0,Banashankari Jayanagar
Kapoor's Cafe,North Indian,4.2,800.0,Banashankari Jayanagar
Jalsa,North Indian Mughlai Chinese,4.1,800.0,Banashankari Banashankari
Hara Fine Dine,North Indian Chinese,4.0,800.0,Banashankari Banashankari
1947,North Indian Chinese,4.0,850.0,Banashankari Banashankari
Mint and Mustard,North Indian Chinese,4.0,750.0,Banashankari Basavanagudi
Pramukh Family Restaurant,North Indian Chinese Mughlai,3.9,700.0,Banashankari Basavanagudi
Thamboola,North Indian Chinese Continental,3.9,800.0,Banashankari Banashankari
